In [ ]:
pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.4 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [26]:
import networkx as nx
import random
import math
import matplotlib.pyplot as plt
import math
import random
import numpy as np
import networkx as nx
from joblib import load
import pickle
from node2vec import Node2Vec
from gensim.models import Word2Vec
from networkx.algorithms import community

In [27]:
def create_Big_Graphs():
  num_nodes = 2000
  BigList_2000 =[]
  random_complete = nx.complete_graph(2000)
  for i in range(2000):
    for j in range(i + 1, 2000):
      if random.random() < 0.5:
        random_complete.remove_edge(i, j)
  BigList_2000.append(random_complete)
  #-------------------------------------------------
  random_multigraph = nx.MultiGraph()
  for i in range(num_nodes):
      random_multigraph.add_node(i)
  for i in range(num_nodes):
      for j in range(i + 1, num_nodes):
          if random.random() < 0.5:
              random_multigraph.add_edge(i, j)
  BigList_2000.append(random_multigraph)
  #-------------------------------------------------
  pseudograph = nx.MultiGraph()
  pseudograph.add_nodes_from(range(num_nodes))
  for node in pseudograph.nodes():
      pseudograph.add_edge(node, node)
      random_nodes = random.sample(pseudograph.nodes(), k=3)
      pseudograph.add_edges_from([(node, random_node) for random_node in random_nodes])
  BigList_2000.append(pseudograph)
  #-------------------------------------------------
  planar_graph = nx.random_geometric_graph(num_nodes, radius=0.2)
  while not nx.is_connected(planar_graph):
      planar_graph = nx.random_geometric_graph(num_nodes, radius=0.2)
  BigList_2000.append(planar_graph)
  #-------------------------------------------------
  def random_hamiltonian_graph(n):
      graph = nx.random_regular_graph(2, n)
      for i in range(n):
          for j in range(i + 1, n):
              if random.random() < 0.5 and not graph.has_edge(i, j):
                  graph.add_edge(i, j)
      return graph
  random_hamiltonian_graph = random_hamiltonian_graph(num_nodes)
  BigList_2000.append(random_hamiltonian_graph)
  return BigList_2000

In [28]:
def create_main_graphs(num_nodes):
  main_list =[]
  random_regular_3 = nx.random_regular_graph(3, num_nodes)
  random_regular_20 = nx.random_regular_graph(20, num_nodes)
  main_list.append(random_regular_3)
  main_list.append(random_regular_20)
  #-------------------------------------------------
  # Create a random connected graph by generating a random spanning tree
  random_connected_tree = nx.random_tree(num_nodes)
  # Add additional edges to maintain connectivity
  while not nx.is_connected(random_connected_tree):
      node1 = random.choice(list(random_connected_tree.nodes()))
      node2 = random.choice(list(random_connected_tree.nodes()))
      if node1 != node2 and not random_connected_tree.has_edge(node1, node2):
          random_connected_tree.add_edge(node1, node2)
  main_list.append(random_connected_tree)
  #-------------------------------------------------
  def generate_random_connected_cycle_graph(num_nodes):
      if num_nodes < 3:
          raise ValueError("Number of nodes must be at least 3 for a cycle.")
      cycle_graph = nx.cycle_graph(num_nodes)
      # Connect the cycle to form a connected cycle graph
      connected_cycle_graph = nx.connected_watts_strogatz_graph(num_nodes, 2, 0.1)
      return connected_cycle_graph
  random_cycle_graph = generate_random_connected_cycle_graph(num_nodes)
  main_list.append(random_cycle_graph)
  #-------------------------------------------------
  def generate_random_connected_barabasi_albert_graph(num_nodes, m):
      if num_nodes <= m:
          raise ValueError("Number of nodes must be greater than m for a Barabási-Albert graph.")
      ba_graph = nx.barabasi_albert_graph(num_nodes, m)
      while not nx.is_connected(ba_graph):
          # Add edges to connect the graph
          non_edges = list(nx.non_edges(ba_graph))
          edge_to_add = non_edges[0]
          ba_graph.add_edge(*edge_to_add)
      return ba_graph
  m_parameter = 2
  random_connected_ba_graph = generate_random_connected_barabasi_albert_graph(num_nodes, m_parameter)
  main_list.append(random_connected_ba_graph)
  #-------------------------------------------------
  def generate_random_connected_erdos_renyi_graph(num_nodes, probability):
      er_graph = nx.erdos_renyi_graph(num_nodes, probability)
      while not nx.is_connected(er_graph):
          # Add edges to connect the graph
          non_edges = list(nx.non_edges(er_graph))
          edge_to_add = non_edges[0]
          er_graph.add_edge(*edge_to_add)
      return er_graph
  edge_probability = 0.1  # Adjust as needed
  random_connected_er_graph = generate_random_connected_erdos_renyi_graph(num_nodes, edge_probability)
  main_list.append(random_connected_er_graph)
  #-------------------------------------------------
  k = 2  # Each node is connected to k nearest neighbors
  p = 0.3  # Probability of rewiring each edge
  random_graph_ws = nx.watts_strogatz_graph(num_nodes, k, p)
  main_list.append(random_graph_ws)
  #-------------------------------------------------
  k = 10  # Each node is connected to k nearest neighbors
  p = 0.3  # Probability of rewiring each edge
  random_graph_ws = nx.watts_strogatz_graph(num_nodes, k, p)
  main_list.append(random_graph_ws)
  #-------------------------------------------------

  def generate_connected_random_geometric_graph(num_nodes, radius):
      random_geo_graph = nx.random_geometric_graph(num_nodes, radius)
      while not nx.is_connected(random_geo_graph):
          # Connect the graph by adding edges
          non_edges = list(nx.non_edges(random_geo_graph))
          random_edge = random.choice(non_edges)
          random_geo_graph.add_edge(*random_edge)
      return random_geo_graph
  radius = 0.1  # Adjust as needed
  connected_random_geo_graph = generate_connected_random_geometric_graph(num_nodes, radius)
  main_list.append(connected_random_geo_graph)
  #-------------------------------------------------
  def generate_connected_random_internet_graph(num_nodes, k, p):
      internet_graph = nx.random_internet_as_graph(num_nodes)
      while not nx.is_connected(internet_graph):
          non_edges = list(nx.non_edges(internet_graph))
          random_edge = non_edges[0]
          internet_graph.add_edge(*random_edge)
      undirected_internet_graph = internet_graph.to_undirected()
      connected_random_internet_graph = nx.connected_watts_strogatz_graph(num_nodes, k, p)
      return connected_random_internet_graph
  k_parameter = 5
  p_parameter = 0.5
  connected_random_internet_graph = generate_connected_random_internet_graph(num_nodes, k_parameter, p_parameter)
  main_list.append(connected_random_internet_graph)
  #-------------------------------------------------
  return main_list


In [29]:
BigGraphs = create_Big_Graphs()
main_list = create_main_graphs(5000)
main_list2 = create_main_graphs(8000)

<ipython-input-27-70e2a112d68a>:24: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_nodes = random.sample(pseudograph.nodes(), k=3)


In [30]:
#Label Graph
def labelling_kernighan(G):
    Y =[]

    #iteration number!!!!!!
    #Dense graph
    if((G.number_of_edges() >= math.comb(G.number_of_nodes(),2)/2) & (G.number_of_edges() <= math.comb(G.number_of_nodes(),2))):
      iteration_number=G.number_of_nodes()
    else:
    #sparse graph
      iteration_number=0.04*G.number_of_nodes()*math.log(G.number_of_nodes())

    partitions = community.kernighan_lin.kernighan_lin_bisection(G, max_iter=int(iteration_number))
    is_equal_partition = (len(list(partitions[0])) > (G.number_of_nodes()/2)-G.number_of_nodes()*0.01) & (len(list(partitions[0])) < (G.number_of_nodes()/2) + G.number_of_nodes() * 0.01) & (len(list(partitions[1])) > (G.number_of_nodes()/2) - G.number_of_nodes()*0.01) & (len(list(partitions[1])) < (G.number_of_nodes()/2) + G.number_of_nodes()*0.01)

    graph_1 = G.subgraph(partitions[0])
    graph_2 = G.subgraph(partitions[1])

    is_connected_partition1 = nx.is_connected(graph_1)
    is_connected_partition2 = nx.is_connected(graph_2)

    if is_connected_partition1 and is_connected_partition2 and is_equal_partition:
        result = True
        Y.append(1)
    else:
        result = False
        Y.append(0)

    return Y

#Feature extraction with Node2vec
def run_n2v(G, dimensions=64, walk_length=80, num_walks=10, p=1, q=1, window=10):

    mdl = Node2Vec(
        G,
        dimensions=dimensions,
        walk_length=walk_length,
        num_walks=num_walks,
        p=p,
        q=q
    )
    mdl = mdl.fit(window=window)

    # Gömme vektörlerini alın
    vectors = [mdl.wv.get_vector(str(x)) for x in G.nodes()]
    return vectors

#Feature extraction with DeepWalk
#We will choose the walk number and walk length by trial and error
def deepwalk(G,num_walks,walk_length):
    # Generate walks for DeepWalk
    walks = []

    for _ in range(num_walks):
        for node in G.nodes():
            walk = [node]
            for _ in range(walk_length - 1):
                neighbors = list(G.neighbors(walk[-1]))
                if neighbors:
                    walk.append(neighbors[0])  # Rastgele bir komşuya ilerleyin
                else:
                    break
            walks.append(walk)

    # Word2Vec modelini eğitin
    model = Word2Vec(walks, vector_size=64, window=5, min_count=0, sg=1, workers=4)

    # Gömme vektörlerini alın
    embedding_vectors = [model.wv.get_vector(node) for node in G.nodes()]

    # Herhangi bir düğümün gömme vektörünü alın

    return embedding_vectors

#median of features of nodes
def median_of_embeddings(G,vectors_array):
    n = G.number_of_nodes()
    final_embedded= []
    temp_embedded = []
    for j in range(vectors_array.shape[1]):
        for i in range(vectors_array.shape[0]):  # Iterate over rows
            element = vectors_array[i][j]
            temp_embedded.append(element)
        temp_embedded.sort()
        median = np.median(temp_embedded)
        final_embedded.append(median)
    return final_embedded

def preprocessing(G):

    while not nx.is_connected(G):
        node1 = random.choice(list(G.nodes()))
        node2 = random.choice(list(G.nodes()))
        if node1 != node2 and not G.has_edge(node1, node2):
            G.add_edge(node1, node2)

    #getting vectors

    vectors_array_deepwalk = np.array(deepwalk(G,10,5))

    embedded_vector = median_of_embeddings(G,vectors_array_deepwalk)

    columns_to_drop_dw = [4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 21, 35, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]
    extracted_vector = np.delete(embedded_vector, columns_to_drop_dw)

    return extracted_vector


In [31]:
# Load your model from .sav file
with open('/content/drive/MyDrive/Colab Notebooks/Team1_last_model.sav','rb') as file:
  model_rf = load(file)


In [32]:
# Input the graph to be predicted and choose graph representation type

for item in BigGraphs:
    data_BigGraphs = preprocessing(item)
    data_BigGraphs = data_BigGraphs.reshape(1,-1)
    label_data_BigGraphs = labelling_kernighan(item)
    print("The BigGraphs label is: ", label_data_BigGraphs)


    predict_Big_Graphs = model_rf.predict(data_BigGraphs)
    print("The predict is ", predict_Big_Graphs)

for item in main_list:
    data_main_list = preprocessing(item)
    data_main_list = data_main_list.reshape(1,-1)
    label_data_main_list = labelling_kernighan(item)
    print("The main_list label is: ", label_data_main_list)

    predict_main_list = model_rf.predict(data_main_list)
    print("The predict is ", predict_main_list)

for item in main_list2:
    data_main_list_2 = preprocessing(item)
    data_main_list_2 = data_main_list_2.reshape(1,-1)
    label_data_main_list_2 = labelling_kernighan(item)
    print("The main_list_2 label is: ", label_data_main_list_2)

    predict_main_list2 = model_rf.predict(data_main_list_2)
    print("The predict is ", predict_main_list2)

The BigGraphs label is:  [1]
The predict is  [0]
The BigGraphs label is:  [1]
The predict is  [1]
The BigGraphs label is:  [1]
The predict is  [1]
The BigGraphs label is:  [1]
The predict is  [1]
The BigGraphs label is:  [1]
The predict is  [1]
The main_list label is:  [1]
The predict is  [1]
The main_list label is:  [1]
The predict is  [1]
The main_list label is:  [0]
The predict is  [0]
The main_list label is:  [0]
The predict is  [1]
The main_list label is:  [1]
The predict is  [1]
The main_list label is:  [1]
The predict is  [1]
The main_list label is:  [0]
The predict is  [0]
The main_list label is:  [1]
The predict is  [0]
The main_list label is:  [1]
The predict is  [1]
The main_list label is:  [1]
The predict is  [0]
The main_list_2 label is:  [1]
The predict is  [0]
The main_list_2 label is:  [1]
The predict is  [0]
The main_list_2 label is:  [0]
The predict is  [0]
The main_list_2 label is:  [0]
The predict is  [0]
The main_list_2 label is:  [1]
The predict is  [1]
The main_l